In [ ]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten,BatchNormalization,Activation,AveragePooling2D,Concatenate
from keras.models import Model
from keras.preprocessing import image
import numpy as np
import pandas as pd
from keras.models import load_model  
from keras.layers import GlobalAveragePooling2D,Flatten
from keras.optimizers import Adam,SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

In [ ]:
from keras.applications import inception_resnet_v2

In [ ]:
from keras.models import load_model

In [ ]:
model = inception_resnet_v2.InceptionResNetV2(weights='imagenet', include_top=False,input_shape=(600,800,3))
#densenet_model = load_model("high_auc_0.8924897119341563checkpoint.h5")

In [ ]:
#model.load_weights("nasnet_large_no_top.h5", by_name=True)
model.summary()

In [ ]:
def conv2d_bn(x,
              filters,
              kernel_size,
              strides=1,
              padding='same',
              activation='relu',
              use_bias=False,
              name=None):
    """Utility function to apply conv + BN.

    # Arguments
        x: input tensor.
        filters: filters in `Conv2D`.
        kernel_size: kernel size as in `Conv2D`.
        strides: strides in `Conv2D`.
        padding: padding mode in `Conv2D`.
        activation: activation in `Conv2D`.
        use_bias: whether to use a bias in `Conv2D`.
        name: name of the ops; will become `name + '_ac'` for the activation
            and `name + '_bn'` for the batch norm layer.

    # Returns
        Output tensor after applying `Conv2D` and `BatchNormalization`.
    """
    x = Conv2D(filters,
              kernel_size,
              strides=strides,
              padding=padding,
              use_bias=use_bias,
              name=name)(x)
    if not use_bias:
        bn_axis = 1 if backend.image_data_format() == 'channels_first' else 3
        bn_name = None if name is None else name + '_bn'
        x = BatchNormalization(axis=bn_axis,
                                      scale=False,
                                      name=bn_name)(x)
    if activation is not None:
        ac_name = None if name is None else name + '_ac'
        x = Activation(activation, name=ac_name)(x)
    return x


In [ ]:
import keras.backend as backend

#尝试添加了一层类似inception的卷积层，降低featuremap大小，通道数不变，效果不太好。

# x = model.layers[-4].output
# #17*23*2080
# x1 = conv2d_bn(x,256,1)
# x1 = conv2d_bn(x1,384,3,padding='valid',strides=2)
# x2 = conv2d_bn(x,256,1)
# x2 = conv2d_bn(x2,384,3,padding='valid',strides=2)
# x2 = conv2d_bn(x2,384,1)
# x3 = conv2d_bn(x,384,1)
# x3 = AveragePooling2D(pool_size=(3,3),padding='valid',strides=2)(x3)
# x4 = conv2d_bn(x,384,1)
# x4 = MaxPooling2D(pool_size=(3,3),padding='valid',strides=2)(x4)

# x = Concatenate(axis=3, name='mixed_lbc')([x1,x2,x3,x4])

x = GlobalAveragePooling2D()(x)

predictions = Dense(2,activation='softmax',name='predictions')(x)

In [ ]:
densenet_model = Model(inputs = model.input, outputs = predictions)

In [ ]:
densenet_model.summary()

In [ ]:
adam = Adam(lr=0.001)
#densenet_model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
import os
import random
print("正常",len(os.listdir("../disk1/train_aug_600_800/normal/")))
print("异常",len(os.listdir("../disk1/train_aug_600_800/bad/")))


In [ ]:
#正常的
#使正负样本平衡
# list_positive = os.listdir("xuelang/crop_data/crop_positive")
# random.shuffle(list_positive)
# list_positive_22758 = list_positive[:22758]
# import shutil
# if "crop_positive_22758" not in os.listdir("xuelang/crop_data"):
#     os.mkdir("xuelang/crop_data/crop_positive_22758")
# for i in list_positive_22758:
#     shutil.copy("xuelang/crop_data/crop_positive/"+i,"xuelang/crop_data/crop_positive_22758/")

In [ ]:
image_generator = ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
        rescale=1./255)
train_generator = image_generator.flow_from_directory(directory='../disk1/train_aug_600_800/',
                                                     target_size=(600,800),
                                                     batch_size=8,
                                                     class_mode='categorical')
val_generator = image_generator.flow_from_directory(directory='xuelang/crop_data/val/',
                                                     target_size=(600,800),
                                                     batch_size=8,
                                                     class_mode='categorical')

In [ ]:
train_generator.class_indices

In [ ]:
import keras.backend as backend
import os
import cv2
import time
class CustomModelCheckpoint(keras.callbacks.Callback): 
    def __init__(self, model, path):
        self.model = model
        self.path = path
        self.best_loss = np.inf
        self.best_auc = 0

    def on_epoch_end(self, epoch, logs=None): 
#         val_loss = logs['val_loss']
#         if val_loss < self.best_loss: 
#             print("\nValidation loss decreased from {} to {}, saving model".format(self.best_loss, val_loss))
#             self.model.save(self.path)
#             self.best_loss = val_loss

        start = time.time()
        
        y_val_predict_all = []
        y_val = []
        #读取验证集normal的一张图像
        for i in os.listdir("xuelang/crop_data/val/normal"):
            img = cv2.imread("xuelang/crop_data/val/normal/" + i)
            #转换为rgb
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#             batch_x = np.zeros((289,) + (224,224,3),backend.floatx())
            img = cv2.resize(img, (800, 600), interpolation=cv2.INTER_AREA)
            batch_x = np.zeros((1,600,800,3),backend.floatx())
            batch_x[0] = img
#             batch_x = np.zeros((6,299,299,3),backend.floatx())
#             x = 0
#             for i in range(3):
#                 y = 0
#                 for j in range(2):
#                     batch_x[i * 2 + j] = cv2.resize(img[y:y + 960, x:x + 960], (299, 299), interpolation=cv2.INTER_CUBIC)
#                     y = y + 960
#                 x = x + 800
            y_val_predict = self.model.predict(batch_x/255.,verbose=0,batch_size=8)
            #取其中可能性最大的为该图片的有问题与否可能性
            #print(y_val_predict)
            y_val_predict = np.max(y_val_predict,axis=0)[0]
            #print(y_val_predict)
            y_val_predict_all.append(y_val_predict)
            #normal 可能性应该为0
            y_val.append(0)
        print("正常的预测完成")
        
        #读取验证集bad的一张图像
        for i in os.listdir("xuelang/crop_data/val/bad"):
            img = cv2.imread("xuelang/crop_data/val/bad/" + i)
            #转换为rgb
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (800, 600), interpolation=cv2.INTER_AREA)
            batch_x = np.zeros((1,600,800,3),backend.floatx())
            batch_x[0] = img
#             x = 0
#             for i in range(3):
#                 y = 0
#                 for j in range(2):
#                     batch_x[i * 2 + j] = cv2.resize(img[y:y + 960, x:x + 960], (299, 299), interpolation=cv2.INTER_CUBIC)
#                     y = y + 960
#                 x = x + 800
#             x = 0
            
            y_val_predict = self.model.predict(batch_x/255.,verbose=0,batch_size=8)
        
            #取其中可能性最大的为该图片的有问题与否可能性
            y_val_predict = np.max(y_val_predict,axis=0)[0]
            y_val_predict_all.append(y_val_predict)
            #bad 可能性应该为1
            y_val.append(1)
        print("瑕疵的预测完成")
        
        print("验证集验证完成")
        
        end = time.time()

        print("用时:",end - start)
        
        
        from sklearn import metrics
        val_auc = metrics.roc_auc_score(y_val,y_val_predict_all)
        print('val_auc:',val_auc)
        if val_auc > self.best_auc or val_auc > 0.9: 
            print("\nValidation auc increased from {} to {}, saving model".format(self.best_auc, val_auc))
            self.model.save('high_auc_'+str(val_auc)+self.path)
            self.best_auc = val_auc
            
            
        abc = np.array(y_val_predict_all)
        abc[abc>=0.5] = 1
        abc[abc<0.5] = 0
        
        print("accuracy:",np.sum(abc == np.array(y_val))/len(y_val))

In [ ]:
from keras.utils import multi_gpu_model
from keras.optimizers import SGD
#model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')


# Replicates `model` on 8 GPUs.
# This assumes that your machine has 8 available GPUs.
parallel_model = multi_gpu_model(densenet_model, gpus=2)
parallel_model.compile(loss='categorical_crossentropy',optimizer=adam,metrics = ['accuracy'])
#parallel_model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy',metrics = ['accuracy'])


In [ ]:
history = parallel_model.fit_generator(train_generator,validation_data=val_generator,verbose=1,epochs=100,callbacks=[CustomModelCheckpoint(densenet_model,'68checkpoint.h5')])